<a href="https://colab.research.google.com/github/EYBishal/Python-Training-Bootcamp/blob/main/23rd-Dec-2025/Exercise-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("Exercise").getOrCreate()

In [44]:
sales_data = [
    ("TXN001","Delhi","Laptop","Electronics","45000","2024-01-05","Completed"),
    ("TXN002","Mumbai","Mobile ","electronics","32000","05/01/2024","Completed"),
    ("TXN003","Bangalore","Tablet"," Electronics ","30000","2024/01/06","Completed"),
    ("TXN004","Delhi","Laptop","Electronics","","2024-01-07","Cancelled"),
    ("TXN005","Chennai","Mobile","Electronics","invalid","2024-01-08","Completed"),
    ("TXN006","Mumbai","Tablet","Electronics",None,"2024-01-08","Completed"),
    ("TXN007","Delhi","Laptop","electronics","45000","09-01-2024","Completed"),
    ("TXN008","Bangalore","Mobile","Electronics","28000","2024-01-09","Completed"),
    ("TXN009","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),
    ("TXN009","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed")
]
customer_data = [
("C001","Delhi","Premium"),
("C002","Mumbai","Standard"),
("C003","Bangalore","Premium"),
("C004","Chennai","Standard"),
("C005","Mumbai","Premium")
]
city_lookup = [
("Delhi","Tier-1"),
("Mumbai","Tier-1"),
("Bangalore","Tier-1"),
("Chennai","Tier-2")
]

In [10]:
from pyspark.sql.types import StructType,StructField,StringType,DateType,IntegerType,LongType
from pyspark.sql.functions import col, lit, when,regexp_replace, split, trim, array_compact, transform, get_json_object, lower

In [4]:
customer_schema=StructType([
    StructField("customer_id",StringType(),True),
    StructField("city",StringType(),True),
    StructField("membership_type",StringType(),True)
])

In [5]:
city_schema=StructType([
    StructField("city",StringType(),True),
    StructField("tier",StringType(),True)
])

In [6]:

sales_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("city", StringType(), True),
    StructField("product",StringType(),True),
    StructField("category",StringType(),True),
    StructField("amount",StringType(),True),
    StructField("date",StringType(),True),
     StructField("status",StringType(),True),
])
df_sales=spark.createDataFrame(data=sales_data,schema=sales_schema)
df_customer=spark.createDataFrame(data=customer_data,schema=customer_schema)
df_city=spark.createDataFrame(data=city_lookup,schema=city_schema)

In [8]:
from pyspark.sql.functions import col,upper,to_date,coalesce,split,lit,array_remove,regexp_extract

In [15]:
numeric_conv=regexp_extract(col("amount"),r"(\d+)",0)
check=df_sales.withColumn("amount",when(((numeric_conv)=="")|numeric_conv.isNull(),lit(0)).otherwise(numeric_conv.cast('int')))
check.withColumn("date",
            coalesce(
                to_date(col("date"),"yyyy-MM-dd"),
                to_date(col("date"),"dd-MM-yyyy"),
                to_date(col("date"),"yyyy/MM/dd")
            ))
check.withColumn("category",upper(trim(col("category"))))
check.withColumn("city",(trim(col("city"))))
check.withColumn("product",(trim(col("product"))))
check.show()

+--------------+---------+-------+-------------+------+----------+---------+
|transaction_id|     city|product|     category|amount|      date|   status|
+--------------+---------+-------+-------------+------+----------+---------+
|        TXN001|   Delhi | Laptop|  Electronics| 45000|2024-01-05|Completed|
|        TXN002|   Mumbai|Mobile |  electronics| 32000|05/01/2024|Completed|
|        TXN003|Bangalore| Tablet| Electronics | 30000|2024/01/06|Completed|
|        TXN004|    Delhi| Laptop|  Electronics|     0|2024-01-07|Cancelled|
|        TXN005|  Chennai| Mobile|  Electronics|     0|2024-01-08|Completed|
|        TXN006|   Mumbai| Tablet|  Electronics|     0|2024-01-08|Completed|
|        TXN007|    Delhi| Laptop|  electronics| 45000|09-01-2024|Completed|
|        TXN008|Bangalore| Mobile|  Electronics| 28000|2024-01-09|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|

In [16]:
check.dropDuplicates(['transaction_id']).show()

+--------------+---------+-------+-------------+------+----------+---------+
|transaction_id|     city|product|     category|amount|      date|   status|
+--------------+---------+-------+-------------+------+----------+---------+
|        TXN001|   Delhi | Laptop|  Electronics| 45000|2024-01-05|Completed|
|        TXN002|   Mumbai|Mobile |  electronics| 32000|05/01/2024|Completed|
|        TXN003|Bangalore| Tablet| Electronics | 30000|2024/01/06|Completed|
|        TXN004|    Delhi| Laptop|  Electronics|     0|2024-01-07|Cancelled|
|        TXN005|  Chennai| Mobile|  Electronics|     0|2024-01-08|Completed|
|        TXN006|   Mumbai| Tablet|  Electronics|     0|2024-01-08|Completed|
|        TXN007|    Delhi| Laptop|  electronics| 45000|09-01-2024|Completed|
|        TXN008|Bangalore| Mobile|  Electronics| 28000|2024-01-09|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
+--------------+---------+-------+-------------+------+----------+---------+

In [17]:
check.filter(col("status")=="Completed").show()

+--------------+---------+-------+-------------+------+----------+---------+
|transaction_id|     city|product|     category|amount|      date|   status|
+--------------+---------+-------+-------------+------+----------+---------+
|        TXN001|   Delhi | Laptop|  Electronics| 45000|2024-01-05|Completed|
|        TXN002|   Mumbai|Mobile |  electronics| 32000|05/01/2024|Completed|
|        TXN003|Bangalore| Tablet| Electronics | 30000|2024/01/06|Completed|
|        TXN005|  Chennai| Mobile|  Electronics|     0|2024-01-08|Completed|
|        TXN006|   Mumbai| Tablet|  Electronics|     0|2024-01-08|Completed|
|        TXN007|    Delhi| Laptop|  electronics| 45000|09-01-2024|Completed|
|        TXN008|Bangalore| Mobile|  Electronics| 28000|2024-01-09|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
+--------------+---------+-------+-------------+------+----------+---------+

In [18]:
from pyspark.sql.functions import broadcast
check.join(broadcast(df_city),on="city",how="left").show()

+---------+--------------+-------+-------------+------+----------+---------+------+
|     city|transaction_id|product|     category|amount|      date|   status|  tier|
+---------+--------------+-------+-------------+------+----------+---------+------+
|   Delhi |        TXN001| Laptop|  Electronics| 45000|2024-01-05|Completed|  NULL|
|   Mumbai|        TXN002|Mobile |  electronics| 32000|05/01/2024|Completed|Tier-1|
|Bangalore|        TXN003| Tablet| Electronics | 30000|2024/01/06|Completed|Tier-1|
|    Delhi|        TXN004| Laptop|  Electronics|     0|2024-01-07|Cancelled|Tier-1|
|  Chennai|        TXN005| Mobile|  Electronics|     0|2024-01-08|Completed|Tier-2|
|   Mumbai|        TXN006| Tablet|  Electronics|     0|2024-01-08|Completed|Tier-1|
|    Delhi|        TXN007| Laptop|  electronics| 45000|09-01-2024|Completed|Tier-1|
|Bangalore|        TXN008| Mobile|  Electronics| 28000|2024-01-09|Completed|Tier-1|
|   Mumbai|        TXN009| Laptop|  Electronics| 55000|2024-01-10|Completed|

In [27]:
from pyspark.sql import functions as f
s=check.groupBy("city").agg(f.sum("amount")).show()

+---------+-----------+
|     city|sum(amount)|
+---------+-----------+
|Bangalore|      58000|
|  Chennai|          0|
|   Mumbai|     142000|
|   Delhi |      45000|
|    Delhi|      45000|
+---------+-----------+



In [25]:
from pyspark.sql import functions as f
check.groupBy("product").agg(f.sum("amount")).show()

+-------+-----------+
|product|sum(amount)|
+-------+-----------+
| Laptop|     200000|
|Mobile |      32000|
| Mobile|      28000|
| Tablet|      30000|
+-------+-----------+



In [41]:



from pyspark.sql import functions as f
from pyspark.sql.window import Window

df_s = check.groupBy("city").agg(f.sum("amount").alias("tot"))

w = Window.orderBy(f.desc("tot"))

df_ranked = df_s.withColumn("rank", f.rank().over(w))

df_ranked.show()


+---------+------+----+
|     city|   tot|rank|
+---------+------+----+
|   Mumbai|142000|   1|
|Bangalore| 58000|   2|
|   Delhi | 45000|   3|
|    Delhi| 45000|   3|
|  Chennai|     0|   5|
+---------+------+----+



In [45]:

from pyspark.sql import functions as f
from pyspark.sql.window import Window

df_s = (
    check.groupBy("city", "product")
         .agg(f.sum("amount").alias("total_revenue"))
)
df_s.show()
w=Window.partitionBy("city").orderBy(f.desc("total_revenue"))
df_ranked=df_s.withColumn("rank",f.rank().over(w))
df_ranked.show()

+---------+-------+-------------+
|     city|product|total_revenue|
+---------+-------+-------------+
|  Chennai| Mobile|            0|
|   Delhi | Laptop|        45000|
|   Mumbai|Mobile |        32000|
|    Delhi| Laptop|        45000|
|Bangalore| Tablet|        30000|
|   Mumbai| Tablet|            0|
|   Mumbai| Laptop|       110000|
|Bangalore| Mobile|        28000|
+---------+-------+-------------+

+---------+-------+-------------+----+
|     city|product|total_revenue|rank|
+---------+-------+-------------+----+
|Bangalore| Tablet|        30000|   1|
|Bangalore| Mobile|        28000|   2|
|  Chennai| Mobile|            0|   1|
|    Delhi| Laptop|        45000|   1|
|   Delhi | Laptop|        45000|   1|
|   Mumbai| Laptop|       110000|   1|
|   Mumbai|Mobile |        32000|   2|
|   Mumbai| Tablet|            0|   3|
+---------+-------+-------------+----+



In [46]:

from pyspark.sql import functions as f
from pyspark.sql.window import Window
daily_city = (
    check
    .groupBy("date", "city")
    .agg(f.sum("amount").alias("total_revenue"))
)
w = Window.partitionBy("date").orderBy(f.desc("total_revenue"))

top_per_day = (
    daily_city
    .withColumn("rank", f.dense_rank().over(w))
    .filter(f.col("rank") == 1)
    .orderBy("date")
)

top_per_day.show()


+----------+---------+-------------+----+
|      date|     city|total_revenue|rank|
+----------+---------+-------------+----+
|05/01/2024|   Mumbai|        32000|   1|
|09-01-2024|    Delhi|        45000|   1|
|2024-01-05|   Delhi |        45000|   1|
|2024-01-07|    Delhi|            0|   1|
|2024-01-08|  Chennai|            0|   1|
|2024-01-08|   Mumbai|            0|   1|
|2024-01-09|Bangalore|        28000|   1|
|2024-01-10|   Mumbai|       110000|   1|
|2024/01/06|Bangalore|        30000|   1|
+----------+---------+-------------+----+



In [48]:
check.cache()
check.show()

+--------------+---------+-------+-------------+------+----------+---------+
|transaction_id|     city|product|     category|amount|      date|   status|
+--------------+---------+-------+-------------+------+----------+---------+
|        TXN001|   Delhi | Laptop|  Electronics| 45000|2024-01-05|Completed|
|        TXN002|   Mumbai|Mobile |  electronics| 32000|05/01/2024|Completed|
|        TXN003|Bangalore| Tablet| Electronics | 30000|2024/01/06|Completed|
|        TXN004|    Delhi| Laptop|  Electronics|     0|2024-01-07|Cancelled|
|        TXN005|  Chennai| Mobile|  Electronics|     0|2024-01-08|Completed|
|        TXN006|   Mumbai| Tablet|  Electronics|     0|2024-01-08|Completed|
|        TXN007|    Delhi| Laptop|  electronics| 45000|09-01-2024|Completed|
|        TXN008|Bangalore| Mobile|  Electronics| 28000|2024-01-09|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|

In [49]:

import time

start_time_cached = time.time()
total_revenue_cached = check.groupBy("city").agg({"amount":"sum"}).collect()
end_time_cached = time.time()
print(f"Time taken with caching: {end_time_cached - start_time_cached:.4f} seconds")

Time taken with caching: 0.2913 seconds


In [51]:

import time
check.unpersist()
start_time_cached = time.time()
total_revenue_cached = check.groupBy("city").agg({"amount":"sum"}).collect()
end_time_cached = time.time()
print(f"Time taken without caching: {end_time_cached - start_time_cached:.4f} seconds")

Time taken without caching: 0.5778 seconds


In [52]:

repartitioned_sales_df = check.repartition('city')
repartitioned_sales_df.cache()
repartitioned_sales_df.show()

+--------------+---------+-------+-------------+------+----------+---------+
|transaction_id|     city|product|     category|amount|      date|   status|
+--------------+---------+-------+-------------+------+----------+---------+
|        TXN003|Bangalore| Tablet| Electronics | 30000|2024/01/06|Completed|
|        TXN008|Bangalore| Mobile|  Electronics| 28000|2024-01-09|Completed|
|        TXN005|  Chennai| Mobile|  Electronics|     0|2024-01-08|Completed|
|        TXN002|   Mumbai|Mobile |  electronics| 32000|05/01/2024|Completed|
|        TXN006|   Mumbai| Tablet|  Electronics|     0|2024-01-08|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
|        TXN009|   Mumbai| Laptop|  Electronics| 55000|2024-01-10|Completed|
|        TXN001|   Delhi | Laptop|  Electronics| 45000|2024-01-05|Completed|
|        TXN004|    Delhi| Laptop|  Electronics|     0|2024-01-07|Cancelled|
|        TXN007|    Delhi| Laptop|  electronics| 45000|09-01-2024|Completed|

In [53]:
repartitioned_sales_df.write.mode("overwrite").parquet("cleaned_data.parquet")


repartitioned_sales_df.write.mode("overwrite").orc("cleaned_orders.orc")

In [54]:
check.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(amount, CASE WHEN 'or('`=`('regexp_extract('amount, (\d+), 0), ), 'isNull('regexp_extract('amount, (\d+), 0))) THEN 0 ELSE cast('regexp_extract('amount, (\d+), 0) as int) END, None)]
+- LogicalRDD [transaction_id#0, city#1, product#2, category#3, amount#4, date#5, status#6], false

== Analyzed Logical Plan ==
transaction_id: string, city: string, product: string, category: string, amount: int, date: string, status: string
Project [transaction_id#0, city#1, product#2, category#3, CASE WHEN ((regexp_extract(amount#4, (\d+), 0) = ) OR isnull(regexp_extract(amount#4, (\d+), 0))) THEN 0 ELSE cast(regexp_extract(amount#4, (\d+), 0) as int) END AS amount#15, date#5, status#6]
+- LogicalRDD [transaction_id#0, city#1, product#2, category#3, amount#4, date#5, status#6], false

== Optimized Logical Plan ==
Project [transaction_id#0, city#1, product#2, category#3, CASE WHEN ((regexp_extract(amount#4, (\d+), 0) = ) OR isnull(regexp_extra

In [55]:
check.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- status: string (nullable = true)



In [56]:
check.count()

10

In [58]:
check.filter(col("amount").isNull()).count()

0

In [60]:
check.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- status: string (nullable = true)



In [59]:
check.describe().show()

+-------+--------------+---------+-------+-------------+------------------+----------+---------+
|summary|transaction_id|     city|product|     category|            amount|      date|   status|
+-------+--------------+---------+-------+-------------+------------------+----------+---------+
|  count|            10|       10|     10|           10|                10|        10|       10|
|   mean|          NULL|     NULL|   NULL|         NULL|           29000.0|      NULL|     NULL|
| stddev|          NULL|     NULL|   NULL|         NULL|22105.806175452337|      NULL|     NULL|
|    min|        TXN001|Bangalore| Laptop| Electronics |                 0|05/01/2024|Cancelled|
|    max|        TXN009|   Mumbai| Tablet|  electronics|             55000|2024/01/06|Completed|
+-------+--------------+---------+-------+-------------+------------------+----------+---------+

